## 1) 데이터셋 전처리

In [2]:
import numpy as np
import pandas as pd
import random

from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, TensorDataset

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device:", device)

device: cpu


In [4]:
# 컬럼 정의 + 데이터 로드

header_names = [
    "duration","protocol_type","service","flag","src_bytes","dst_bytes","land",
    "wrong_fragment","urgent","hot","num_failed_logins","logged_in","num_compromised",
    "root_shell","su_attempted","num_root","num_file_creations","num_shells",
    "num_access_files","num_outbound_cmds","is_host_login","is_guest_login","count",
    "srv_count","serror_rate","srv_serror_rate","rerror_rate","srv_rerror_rate",
    "same_srv_rate","diff_srv_rate","srv_diff_host_rate","dst_host_count",
    "dst_host_srv_count","dst_host_same_srv_rate","dst_host_diff_srv_rate",
    "dst_host_same_src_port_rate","dst_host_srv_diff_host_rate","dst_host_serror_rate",
    "dst_host_srv_serror_rate","dst_host_rerror_rate","dst_host_srv_rerror_rate",
    "attack_type","difficulty"
]

train_df = pd.read_csv("NSL-KDD_datasets/nsl-kdd/KDDTrain+.txt", names=header_names)
test_df  = pd.read_csv("NSL-KDD_datasets/nsl-kdd/KDDTest+.txt",  names=header_names)

In [5]:
# attack_type -> attack_category 매핑

dos = {"back","land","neptune","pod","smurf","teardrop","apache2","udpstorm","processtable","mailbomb"}
probe = {"satan","ipsweep","nmap","portsweep","mscan","saint"}
r2l = {"guess_passwd","ftp_write","imap","phf","multihop","warezmaster","warezclient",
       "spy","xlock","xsnoop","snmpguess","snmpgetattack","httptunnel","sendmail","named"}
u2r = {"buffer_overflow","loadmodule","perl","rootkit","ps","sqlattack","xterm"}

def map_attack(x):
    if x in ["normal","benign"]: return "benign"
    if x in dos: return "dos"
    if x in probe: return "probe"
    if x in r2l: return "r2l"
    if x in u2r: return "u2r"
    return "attack"

train_df["attack_category"] = train_df["attack_type"].apply(map_attack)
test_df["attack_category"]  = test_df["attack_type"].apply(map_attack)

In [6]:
# 불필요한 컬럼 제거

drop_cols = ["attack_type", "difficulty", "num_outbound_cmds"]

train_df.drop(columns=drop_cols, errors="ignore", inplace=True)
test_df.drop(columns=drop_cols, errors="ignore", inplace=True)

In [7]:
# X, y 분리

y_train_raw = train_df["attack_category"]
y_test_raw  = test_df["attack_category"]

X_train_raw = train_df.drop(columns=["attack_category"])
X_test_raw  = test_df.drop(columns=["attack_category"])

In [8]:
# 원-핫 인코딩 (train/test 컬럼 정렬)

all_X = pd.concat([X_train_raw, X_test_raw], axis=0)

cat_cols = all_X.select_dtypes(include=["object"]).columns
all_X = pd.get_dummies(all_X, columns=cat_cols, drop_first=True)

X_train = all_X.iloc[:len(X_train_raw)]
X_test  = all_X.iloc[len(X_train_raw):]

In [9]:
# 스케일링

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test  = scaler.transform(X_test)

In [10]:
# 라벨 인코딩

le = LabelEncoder()
le.fit(pd.concat([y_train_raw, y_test_raw], axis=0))

y_train = le.transform(y_train_raw).astype(np.int64)
y_test  = le.transform(y_test_raw).astype(np.int64)

num_classes = len(le.classes_)
print("classes:", le.classes_)

classes: ['attack' 'benign' 'dos' 'probe' 'r2l' 'u2r']


## GA Feature Selection

In [11]:
NUM_FEATURES = X_train.shape[1]
POP_SIZE = 30
N_GEN = 20
CROSS_RATE = 0.8
MUT_RATE = 0.02

def random_chromosome():
    return np.random.randint(0, 2, size=NUM_FEATURES)

def fitness(chromosome):
    selected_idx = np.where(chromosome == 1)[0]
    if len(selected_idx) == 0:
        return 0.0

    X_sel = X_train[:, selected_idx]
    y_sel = y_train

    X_sel = X_sel.reshape(X_sel.shape[0], 1, X_sel.shape[1])

    X_t = torch.tensor(X_sel, dtype=torch.float32)
    y_t = torch.tensor(y_sel, dtype=torch.long)

    dataset = TensorDataset(X_t, y_t)
    loader = DataLoader(dataset, batch_size=256, shuffle=True)

    lstm = nn.LSTM(
        input_size=X_sel.shape[2],
        hidden_size=32,
        batch_first=True
    )
    fc = nn.Linear(32, num_classes)

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(
        list(lstm.parameters()) + list(fc.parameters()), lr=1e-3
    )

    lstm.train()
    for _ in range(2):  # GA 내부는 짧게
        for xb, yb in loader:
            out, _ = lstm(xb)
            logits = fc(out[:, -1, :])
            loss = criterion(logits, yb)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

    lstm.eval()
    with torch.no_grad():
        out, _ = lstm(X_t)
        logits = fc(out[:, -1, :])
        preds = torch.argmax(logits, dim=1)
        acc = (preds == y_t).float().mean().item()

    return acc

In [12]:
population = [random_chromosome() for _ in range(POP_SIZE)]

for gen in range(N_GEN):
    scores = [fitness(c) for c in population]
    best_idx = np.argmax(scores)
    best_chrom = population[best_idx]

    print(f"Gen {gen} | Best acc: {scores[best_idx]:.4f} | Features: {best_chrom.sum()}")

    parents = random.choices(population, weights=scores, k=POP_SIZE)

    new_pop = []
    for i in range(0, POP_SIZE, 2):
        p1, p2 = parents[i], parents[i+1]
        if random.random() < CROSS_RATE:
            point = random.randint(1, NUM_FEATURES - 1)
            c1 = np.concatenate([p1[:point], p2[point:]])
            c2 = np.concatenate([p2[:point], p1[point:]])
        else:
            c1, c2 = p1.copy(), p2.copy()

        for c in [c1, c2]:
            for j in range(NUM_FEATURES):
                if random.random() < MUT_RATE:
                    c[j] ^= 1
            new_pop.append(c)

    population = new_pop

Gen 0 | Best acc: 0.9767 | Features: 64
Gen 1 | Best acc: 0.9789 | Features: 66
Gen 2 | Best acc: 0.9790 | Features: 66
Gen 3 | Best acc: 0.9787 | Features: 72
Gen 4 | Best acc: 0.9777 | Features: 69
Gen 5 | Best acc: 0.9795 | Features: 71
Gen 6 | Best acc: 0.9806 | Features: 69
Gen 7 | Best acc: 0.9817 | Features: 65
Gen 8 | Best acc: 0.9800 | Features: 58
Gen 9 | Best acc: 0.9794 | Features: 66
Gen 10 | Best acc: 0.9767 | Features: 63
Gen 11 | Best acc: 0.9790 | Features: 52
Gen 12 | Best acc: 0.9775 | Features: 56
Gen 13 | Best acc: 0.9787 | Features: 58
Gen 14 | Best acc: 0.9785 | Features: 56
Gen 15 | Best acc: 0.9787 | Features: 62
Gen 16 | Best acc: 0.9784 | Features: 55
Gen 17 | Best acc: 0.9788 | Features: 57
Gen 18 | Best acc: 0.9794 | Features: 56
Gen 19 | Best acc: 0.9801 | Features: 59


In [13]:
best_features = np.where(best_chrom == 1)[0]
print("Selected features:", len(best_features))

Selected features: 59


In [14]:
# LSTM 입력 형태로 변환

X_train_ga = X_train[:, best_features]
X_test_ga  = X_test[:, best_features]

X_train_lstm = X_train_ga.reshape(X_train_ga.shape[0], 1, X_train_ga.shape[1])
X_test_lstm  = X_test_ga.reshape(X_test_ga.shape[0], 1, X_test_ga.shape[1])

y_train = y_train.astype(np.int64)
y_test  = y_test.astype(np.int64)

## 2) Dataset / DataLoader

In [15]:
class NslKddDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.from_numpy(X).float()   # 🔥 핵심
        self.y = torch.from_numpy(y).long()    # 🔥 핵심

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

batch_size = 256

train_ds = NslKddDataset(X_train_lstm, y_train)
test_ds  = NslKddDataset(X_test_lstm,  y_test)

train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, drop_last=False)
test_loader  = DataLoader(test_ds,  batch_size=batch_size, shuffle=False, drop_last=False)

## 3) LSTM 분류 모델 정의 (최소 구조)

In [16]:
class LSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes, num_layers=1, dropout=0.2):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0.0
        )
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # x: (B, T, F)
        out, (hn, cn) = self.lstm(x)
        # hn: (num_layers, B, hidden_size) -> 마지막 레이어의 hidden 사용
        h_last = hn[-1]                 # (B, hidden_size)
        h_last = self.dropout(h_last)
        logits = self.fc(h_last)        # (B, num_classes)
        return logits

## 4) 불균형 대응: class_weight로 loss 가중치 주기 (SMOTE 대신 추천)

In [17]:
classes_in_train = np.unique(y_train)  # y_train에 실제로 존재하는 라벨만
w = compute_class_weight(
    class_weight="balanced",
    classes=classes_in_train,
    y=y_train
).astype(np.float32)

# CrossEntropyLoss는 weight 길이가 num_classes여야 하므로 전체 길이로 확장
w_full = np.ones(num_classes, dtype=np.float32)
w_full[classes_in_train] = w

class_weights_t = torch.tensor(w_full, dtype=torch.float32)

In [18]:
num_features = X_train_lstm.shape[2]   # (N, 1, F)에서 F

print(num_features)

59


## 5) 학습 루프 (GPU 자동 사용)

In [19]:
model = LSTMClassifier(
    input_size=num_features,
    hidden_size=64,
    num_classes=num_classes,
    num_layers=1,
    dropout=0.2
).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

criterion = nn.CrossEntropyLoss(weight=class_weights_t.to(device))

def train_one_epoch(model, loader):
    model.train()
    total_loss, total_correct, total = 0.0, 0, 0

    for Xb, yb in loader:
        Xb, yb = Xb.to(device), yb.to(device)

        optimizer.zero_grad()
        logits = model(Xb)
        loss = criterion(logits, yb)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * Xb.size(0)
        preds = torch.argmax(logits, dim=1)
        total_correct += (preds == yb).sum().item()
        total += Xb.size(0)

    return total_loss / total, total_correct / total

@torch.no_grad()
def evaluate(model, loader):
    model.eval()
    total_loss, total_correct, total = 0.0, 0, 0
    all_preds, all_y = [], []

    for Xb, yb in loader:
        Xb, yb = Xb.to(device), yb.to(device)
        logits = model(Xb)
        loss = criterion(logits, yb)

        total_loss += loss.item() * Xb.size(0)
        preds = torch.argmax(logits, dim=1)

        total_correct += (preds == yb).sum().item()
        total += Xb.size(0)

        all_preds.append(preds.cpu().numpy())
        all_y.append(yb.cpu().numpy())

    all_preds = np.concatenate(all_preds)
    all_y = np.concatenate(all_y)
    return total_loss / total, total_correct / total, all_y, all_preds

In [20]:
epochs = 15
best_val_acc = 0.0
best_state = None

for epoch in range(1, epochs + 1):
    tr_loss, tr_acc = train_one_epoch(model, train_loader)
    te_loss, te_acc, y_true, y_pred = evaluate(model, test_loader)

    print(f"[{epoch:02d}/{epochs}] "
          f"train loss={tr_loss:.4f}, acc={tr_acc:.4f} | "
          f"test loss={te_loss:.4f}, acc={te_acc:.4f}")

    if te_acc > best_val_acc:
        best_val_acc = te_acc
        best_state = {k: v.cpu().clone() for k, v in model.state_dict().items()}

print("Best test acc:", best_val_acc)

[01/15] train loss=0.5672, acc=0.8862 | test loss=1.8015, acc=0.7957
[02/15] train loss=0.1983, acc=0.9436 | test loss=2.0548, acc=0.7942
[03/15] train loss=0.1726, acc=0.9530 | test loss=2.2815, acc=0.7913
[04/15] train loss=0.1540, acc=0.9551 | test loss=2.5234, acc=0.8086
[05/15] train loss=0.1352, acc=0.9579 | test loss=2.7880, acc=0.8080
[06/15] train loss=0.1241, acc=0.9598 | test loss=2.7304, acc=0.8071
[07/15] train loss=0.1235, acc=0.9605 | test loss=2.8699, acc=0.8058
[08/15] train loss=0.1149, acc=0.9625 | test loss=3.1172, acc=0.8001
[09/15] train loss=0.1073, acc=0.9654 | test loss=3.1554, acc=0.8017
[10/15] train loss=0.1138, acc=0.9648 | test loss=3.2347, acc=0.8048
[11/15] train loss=0.1070, acc=0.9664 | test loss=3.3352, acc=0.7995
[12/15] train loss=0.1057, acc=0.9678 | test loss=3.4642, acc=0.7982
[13/15] train loss=0.1032, acc=0.9694 | test loss=3.5773, acc=0.8027
[14/15] train loss=0.1007, acc=0.9702 | test loss=3.5520, acc=0.8015
[15/15] train loss=0.0980, acc=0.9

## 6) 최종 평가 (리포트 + 혼동행렬)

In [23]:
# best 모델 복원
if best_state is not None:
    model.load_state_dict(best_state)

_, _, y_true, y_pred = evaluate(model, test_loader)

print(classification_report(y_true, y_pred, target_names=le.classes_))

              precision    recall  f1-score   support

      attack       0.00      0.00      0.00         2
      benign       0.76      0.94      0.84      9711
         dos       0.94      0.84      0.89      7458
       probe       0.79      0.77      0.78      2421
         r2l       0.92      0.32      0.48      2885
         u2r       0.07      0.54      0.13        67

    accuracy                           0.81     22544
   macro avg       0.58      0.57      0.52     22544
weighted avg       0.84      0.81      0.80     22544



c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(averag

In [26]:
# 클래스 이름 확인
classes = le.classes_

# 'attack' 클래스 인덱스 찾기
attack_idx = np.where(classes == 'attack')[0][0]

# 혼동행렬 계산
cm = confusion_matrix(y_true, y_pred)

# attack 행/열 제거
cm_reduced = np.delete(cm, attack_idx, axis=0)  # row 제거
cm_reduced = np.delete(cm_reduced, attack_idx, axis=1)  # column 제거

# attack 제외한 클래스 이름
classes_reduced = np.delete(classes, attack_idx)

# 보기 좋게 DataFrame으로 출력
cm_df = pd.DataFrame(
    cm_reduced,
    index=[f"True_{c}" for c in classes_reduced],
    columns=[f"Pred_{c}" for c in classes_reduced]
)

In [27]:
print(cm_df)

             Pred_benign  Pred_dos  Pred_probe  Pred_r2l  Pred_u2r
True_benign         9124        74         407        60        46
True_dos            1118      6265          74         0         1
True_probe           188       315        1871        15        32
True_r2l            1554         4          26       933       368
True_u2r              23         0           0         8        36
